In [1]:
from torch import nn
import torch
import torch.nn.functional as F

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
import os
import pandas as pd
import numpy as np
import random
import torch.nn.init as init
import pdb
from sklearn.metrics import mean_absolute_percentage_error
torch.autograd.set_detect_anomaly(True)
seed = 42

def file_name(file_dir,file_type='.csv'):#默认为文件夹下的所有文件
    lst = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if(file_type == ''):
                lst.append(file)
            else:
                if os.path.splitext(file)[1] == str(file_type):#获取指定类型的文件名
                    lst.append(file)
    return lst

def normalize0(inputs):
    normalized = []
    for eq in inputs:
        maks = np.max(np.abs(eq))
        if maks != 0:
            normalized.append(eq / maks)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize1(inputs):
    normalized = []
    for eq in inputs:
        mean = np.mean(eq)
        std = np.std(eq)
        if std != 0:
            normalized.append((eq - mean) / std)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize(inputs):
    normalized = []
    for eq in inputs:
        with np.errstate(invalid='ignore'):
            eps = 1e-10  # 可以根据需要调整epsilon的值

            eq_log = [np.log(x + eps) if i < 5 else x for i, x in enumerate(eq)]

            #eq_log = [np.log(x) if i < 5 else x for i, x in enumerate(eq)]
            eq_log1 = np.nan_to_num(eq_log).tolist()
            normalized.append(eq_log1)
    return np.array(normalized)


def k_fold_split(inputs, targets, K, seed=None):
    # 确保所有随机操作都使用相同的种子
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

    ind = int(len(inputs) / K)
    inputsK = []
    targetsK = []

    for i in range(0, K - 1):
        inputsK.append(inputs[i * ind:(i + 1) * ind])
        targetsK.append(targets[i * ind:(i + 1) * ind])

    inputsK.append(inputs[(i + 1) * ind:])
    targetsK.append(targets[(i + 1) * ind:])

    return inputsK, targetsK


def merge_splits(inputs, targets, k, K):
    if k != 0:
        z = 0
        inputsTrain = inputs[z]
        targetsTrain = targets[z]
    else:
        z = 1
        inputsTrain = inputs[z]
        targetsTrain = targets[z]

    for i in range(z + 1, K):
        if i != k:
            inputsTrain = np.concatenate((inputsTrain, inputs[i]))
            targetsTrain = np.concatenate((targetsTrain, targets[i]))

    return inputsTrain, targetsTrain, inputs[k], targets[k]


def targets_to_list(targets):
    targetList = np.array(targets)

    return targetList

In [3]:
# 将输入特征张量 x 根据邻接矩阵 A 进行乘法操作
class nconv(nn.Module):
    def __init__(self):
        super(nconv, self).__init__()

    def forward(self, x, A):
        x = torch.einsum('ncvl,vw->ncwl', (x, A))
        return x.contiguous()

In [36]:
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903 
    图注意力层
    input: (B,N,C_in)
    output: (B,N,C_out)
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features   # 节点表示向量的输入特征数
        self.out_features = out_features   # 节点表示向量的输出特征数
        self.dropout = dropout    # dropout参数
        self.alpha = alpha     # leakyrelu激活的参数
        self.concat = concat   # 如果为true, 再进行elu激活
        
        # 定义可训练参数，即论文中的W和a
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))  
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # 初始化
        self.A = nn.Parameter(torch.zeros(size=(2*out_features, 16)))
        nn.init.xavier_uniform_(self.A.data, gain=1.414)   # 初始化
        
        # 定义leakyrelu激活函数
        self.leakyrelu = nn.LeakyReLU(self.alpha)
    
    def forward(self, inp, adj):
        """
        inp: input_fea [B,N, in_features]  in_features表示节点的输入特征向量元素个数
        adj: 图的邻接矩阵  [N, N] 非零即一，数据结构基本知识
        """
        h = torch.matmul(inp.double(), self.W.double())   # [B, N, out_features]
        N = h.size()[1]    # N 图的节点数

        a_input = torch.cat([h.repeat(1,1,N).view(-1, N*N, self.out_features), h.repeat(1, N, 1)], dim=-1).view(-1, N, N, 2*self.out_features)
        # [B, N, N, 2*out_features]
      
        E = []
        for i in range(16):
            a = self.A[:,i].unsqueeze(1)
            e_col = torch.matmul(a_input.double(), a.double()).squeeze(3)[:,:,i]
            E.append(e_col)

        e = self.leakyrelu(torch.stack(E, dim=2))
        # print(e.shape)

        # [B, N, N, 1] => [B, N, N] 图注意力的相关系数（未归一化）
        
        zero_vec = -1e12 * torch.ones_like(e)    # 将没有连接的边置为负无穷


        attention = torch.where(adj>0, e, zero_vec)   # [B, N, N]
        # 表示如果邻接矩阵元素大于0时，则两个节点有连接，该位置的注意力系数保留，
        # 否则需要mask并置为非常小的值，原因是softmax的时候这个最小值会不考虑。
        attention = F.softmax(attention, dim=1)    # softmax形状保持不变 [B, N, N]，得到归一化的注意力权重！
        # print(attention.shape)
        attention = F.dropout(attention, self.dropout, training=self.training)   # dropout，防止过拟合
        h_prime = torch.matmul(attention, h)  # [B, N, N].[B, N, out_features] => [B, N, out_features]
        # 得到由周围节点通过注意力权重进行更新的表示
        if self.concat:
            return F.relu(h_prime)
        else:
            return h_prime 
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [47]:
class GAT(nn.Module):
    def __init__(self, n_feat, n_hid, n_class, dropout, alpha, n_heads):
        """Dense version of GAT
        n_heads 表示有几个GAL层，最后进行拼接在一起，类似self-attention
        从不同的子空间进行抽取特征。
        """
        super(GAT, self).__init__()
        self.dropout = dropout 
        
        # 定义multi-head的图注意力层
        self.attentions = [GraphAttentionLayer(n_feat, n_hid, dropout=dropout, alpha=alpha, concat=True) for _ in range(n_heads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)   # 加入pytorch的Module模块
        # 输出层，也通过图注意力层来实现，可实现分类、预测等功能
        self.out_att = GraphAttentionLayer(n_hid * n_heads, n_class, dropout=dropout,alpha=alpha, concat=False)
    
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)   # dropout，防止过拟合
        x = torch.cat([att(x, adj) for att in self.attentions], dim=2)  # 将每个head得到的表示进行拼接
        x = F.dropout(x, self.dropout, training=self.training)   # dropout，防止过拟合
        x = self.out_att(x, adj)   # 输出并激活
        #x = F.log_softmax(x, dim=2)[:, -1, :]
        # print(x)
        # print(x)
        # print(x.shape)
        return x[:, -1, :] # log_softmax速度变快，保持数值稳定

    

def train(model, train_loader,hyperparams):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.0015,weight_decay=5e-4)
    criterion = nn.MSELoss()
    optimizer.zero_grad()
    loss = 0
    for inputs, graph_input, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.squeeze(dim=1), graph_input)
        # print('train:',outputs)
        batch_loss = criterion(outputs, targets)
        loss += batch_loss.item()
        batch_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1) # 梯度裁剪
        optimizer.step()
    return loss / len(train_loader)

def val(model, val_loader):
    model.eval()
    criterion = nn.MSELoss()
    val_loss = 0
    with torch.no_grad():
        for val_inputs, val_graph_input, val_targets in val_loader:
            val_outputs = model(val_inputs, val_graph_input)
            # print('val:',val_outputs)
            batch_loss = criterion(val_outputs.squeeze(dim=1), val_targets)
            val_loss += batch_loss.item()
    return val_loss / len(val_loader)
            

def predict(model, test_loader):
    model.eval()
    predictions = []
    for test_inputs, test_graph_input, _ in test_loader:
        batch_predictions = model(test_inputs, test_graph_input)
        predictions.append(batch_predictions)
    predictions = torch.cat(predictions)
    return predictions

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming your PyTorch model is defined as before


if __name__ == "__main__":
    lag_bin = 3
    lag_day = 3
    num_nodes = (int(lag_bin)+1)*(int(lag_day)+1)
    forecast_days = 15
    bin_num=24
    random_state_here = 88
    mape_list = []
    data_dir = './data/volume/0308/'
    files =file_name('./data/')
    stocks_info = list(set(s.split('_25')[0] for s in files))
    print(stocks_info)
    num_epochs=300
    patience=100
    for stock_info in stocks_info[0:2]:
        print(f'>>>>>>>>>>>>>>>>>>>>{stock_info}>>>>>>>>>>>>>>>>>>>>>>>')
        data_dir1 = f'{data_dir}{stock_info}_{lag_bin}_{lag_day}'
        test_set_size = bin_num*forecast_days
        K = 5
        inputs_data = np.load(f'{data_dir1}_inputs.npy', allow_pickle=True)
        inputs_data = [[[torch.tensor(x, dtype=torch.float64) for x in sublist] for sublist in list1] for list1 in inputs_data]
        array_data = np.array(inputs_data)
        inputs = np.reshape(array_data, (len(inputs_data), num_nodes,-1))
        targets = np.load(f'{data_dir1}_output.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.load(f'{data_dir1}_graph_input.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.array([graph_input] * inputs.shape[0])
        graph_features = np.load(f'{data_dir1}_graph_coords.npy', allow_pickle=True).astype(np.float64)
        graph_features = np.array([graph_features] * inputs.shape[0])

        trainInputs, testInputs, traingraphInput, testgraphInput, traingraphFeature, testgraphFeature, trainTargets, testTargets = train_test_split(inputs, graph_input, graph_features, targets, test_size=test_set_size, 
                                                     random_state=random_state_here)
        testInputs = normalize(testInputs)
        # testInputs = test_inputs
        inputsK, targetsK = k_fold_split(trainInputs, trainTargets, K)

        mape_list = []

        test_dataset = TensorDataset(torch.tensor(testInputs), torch.tensor(testgraphInput), torch.tensor(testTargets))
        test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
        K = 5  # Number of folds
        for k in range(K):
            torch.manual_seed(0)  # Set a random seed for reproducibility

            trainInputsAll, trainTargets, valInputsAll, valTargets = merge_splits(inputsK, targetsK, k, K)

            trainGraphInput = traingraphInput[0:trainInputsAll.shape[0], :]
            trainGraphFeatureInput = traingraphFeature[0:trainInputsAll.shape[0], :]

            valGraphInput = traingraphInput[0:valInputsAll.shape[0], :]
            valGraphFeatureInput = traingraphFeature[0:valInputsAll.shape[0], :]

            trainInputs = normalize(trainInputsAll[:, :])
            valInputs = normalize(valInputsAll[:, :])

            # Assuming trainInputs, trainGraphInput, trainGraphFeatureInput, trainTargets are PyTorch tensors
            train_dataset = TensorDataset(torch.tensor(trainInputs), torch.tensor(trainGraphInput),torch.tensor(trainTargets))
            val_dataset = TensorDataset(torch.tensor(valInputs), torch.tensor(valGraphInput),torch.tensor(valTargets))

            # train_dataset = TensorDataset(torch.tensor(trainInputs, dtype=torch.float32), torch.tensor(trainGraphInput, dtype=torch.float32), torch.tensor(trainGraphFeatureInput, dtype=torch.float32), torch.tensor(trainTargets, dtype=torch.float32))
            # val_dataset = TensorDataset(torch.tensor(valInputs, dtype=torch.float32), torch.tensor(valGraphInput, dtype=torch.float32), torch.tensor(valGraphFeatureInput, dtype=torch.float32), torch.tensor(valTargets, dtype=torch.float32))

            train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)

        
            model = GAT(7,8,1, 0.3, 1,2)
            best_val_loss = torch.tensor(float('inf'), dtype=torch.double)
            patience_counter = 0
            optimizer = optim.Adam(model.parameters(), lr=0.0015,weight_decay=5e-4)
            criterion = nn.MSELoss()
            
            for epoch in range(num_epochs):
                loss = train(model, train_loader)
                val_loss = val(model, val_loader)
                if (epoch + 1) % 2 == 0:
                    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')
                # Early stopping
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f'Early stopping at epoch {epoch+1}')
                        break

            predictions=predict(model, test_loader)
            torch.save(model.state_dict(), f'models/gat_{stock_info}_{lag_bin}_{lag_day}_gcn_model_iteration_{k}.pt')
    
            print()
            print('Fold number:', k)

            new_predictions = np.array([item.detach().numpy() for item in predictions]).flatten()
            MAPE = []

            MAPE.append(mean_absolute_percentage_error(testTargets[:], new_predictions[:]))
            print(new_predictions)
            print(MAPE)
            testTargets0 = list(testTargets)

            res = {
                'testTargets': testTargets0,
                'new_predictions': new_predictions
            }

            res_df = pd.DataFrame(res)
            res_df.to_csv(f'./result/gat_{stock_info}_{lag_bin}_{lag_day}_res_test_MAPE{k}.csv', index=False)

            print('MAPE = ', np.array(MAPE).mean())
            MAPE_mean = np.array(MAPE).mean()
            mape_list.append(MAPE)

        print('-')
        print('mape score = ', mape_list)



['603359_XSHG', '603095_XSHG', '000046_XSHE', '000753_XSHE', '600622_XSHG', '000998_XSHE', '300540_XSHE', '300343_XSHE', '002282_XSHE', '300174_XSHE', '000951_XSHE', '603053_XSHG', '002882_XSHE', '300263_XSHE', '002841_XSHE', '300133_XSHE', '002679_XSHE', '300433_XSHE']
>>>>>>>>>>>>>>>>>>>>603359_XSHG>>>>>>>>>>>>>>>>>>>>>>>
Epoch [2/300], Train Loss: 3286678781756.6660, Val Loss: 538377106009.7814
Epoch [4/300], Train Loss: 747273477548.8772, Val Loss: 141929114444.0784
Epoch [6/300], Train Loss: 213168530764.4167, Val Loss: 23675547993.2062
Epoch [8/300], Train Loss: 44723553268.2044, Val Loss: 8495704852.2178
Epoch [10/300], Train Loss: 18288544219.6943, Val Loss: 10756284856.6598
Epoch [12/300], Train Loss: 15049878411.5586, Val Loss: 10077934100.8120
Epoch [14/300], Train Loss: 14486733179.8047, Val Loss: 9734684719.2123
Epoch [16/300], Train Loss: 14191560933.4112, Val Loss: 8607864839.2934
Epoch [18/300], Train Loss: 14660940121.7720, Val Loss: 8530565337.7319
Epoch [20/300], Tra

/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/300], Train Loss: 3163062891234.0239, Val Loss: 485972047594.4319
Epoch [4/300], Train Loss: 763199707255.8934, Val Loss: 105225884863.6493
Epoch [6/300], Train Loss: 225385145976.5112, Val Loss: 28594230780.8580
Epoch [8/300], Train Loss: 64389730962.3696, Val Loss: 16312614734.7734
Epoch [10/300], Train Loss: 20613954827.3697, Val Loss: 9404573621.8939
Epoch [12/300], Train Loss: 15213917713.3639, Val Loss: 9312568413.3236
Epoch [14/300], Train Loss: 16063955310.6554, Val Loss: 11895492648.2221
Epoch [16/300], Train Loss: 16266544600.9591, Val Loss: 11201035217.0159
Epoch [18/300], Train Loss: 16547528468.5113, Val Loss: 9985235265.1630
Epoch [20/300], Train Loss: 14749533829.7326, Val Loss: 9344567457.4802
Epoch [22/300], Train Loss: 14639011392.8350, Val Loss: 6795987652.6507
Epoch [24/300], Train Loss: 14888624433.1218, Val Loss: 8764204522.4808
Epoch [26/300], Train Loss: 15882541320.7755, Val Loss: 9750330075.1395
Epoch [28/300], Train Loss: 16030823165.6023, Val Loss: 

/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/300], Train Loss: 2920001628524.2837, Val Loss: 679940952163.9865
Epoch [4/300], Train Loss: 679005737666.2915, Val Loss: 116381442502.7744
Epoch [6/300], Train Loss: 230927137945.7052, Val Loss: 33882853885.8984
Epoch [8/300], Train Loss: 62137955038.0596, Val Loss: 14414511600.3306
Epoch [10/300], Train Loss: 20260880483.1250, Val Loss: 11092257780.5148
Epoch [12/300], Train Loss: 14990942136.7204, Val Loss: 12545762789.9857
Epoch [14/300], Train Loss: 14453160737.6271, Val Loss: 10251059624.6563
Epoch [16/300], Train Loss: 14432123227.7560, Val Loss: 9332421850.1904
Epoch [18/300], Train Loss: 14601556636.6214, Val Loss: 9568426857.8472
Epoch [20/300], Train Loss: 14604773985.1542, Val Loss: 12490824532.1587
Epoch [22/300], Train Loss: 14184807304.8029, Val Loss: 10947144078.9633
Epoch [24/300], Train Loss: 14460258523.4584, Val Loss: 9927694612.4449
Epoch [26/300], Train Loss: 14334721840.2279, Val Loss: 8579341093.4288
Epoch [28/300], Train Loss: 13962388329.3800, Val Los

/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/300], Train Loss: 2682053591630.3643, Val Loss: 703363971228.9657
Epoch [4/300], Train Loss: 802625714085.4012, Val Loss: 193841117100.9097
Epoch [6/300], Train Loss: 169886308675.1985, Val Loss: 48427283529.8082


KeyboardInterrupt: 